In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import roc_auc_score
#from keras.utils import to_categorical
#from keras.optimizers import Adam, Adadelta
#from keras.models import Sequential
#from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten, Conv1D
#from keras.layers.convolutional import MaxPooling2D, MaxPooling1D
#from keras.models import load_model
from time import time, strftime
import theano
import lasagne
from lasagne import layers
from lasagne import nonlinearities
from lasagne import init
#from lasagne.layers import cuda_convnet
from lasagne.layers import conv
from lasagne.layers import pool
import theano.tensor as T

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Gebruiker\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
def load_subject_train(subj_id):
    data_list, events_list = [], []
    for series_id in range(1, 2):
        fname_data = 'C:/Users/Gebruiker/Documents/Untitled Folder/train/train/subj%d_series%d_data.csv' % (
            subj_id, series_id)
        fname_events = 'C:/Users/Gebruiker/Documents/Untitled Folder/train/train/subj%d_series%d_events.csv' % (
            subj_id, series_id)

        data = pd.read_csv(fname_data)[:10000]
        events = pd.read_csv(fname_events)[:10000]
        channel_names = data.columns[1:]
        data_list.append(data[channel_names].values.T)

        event_names = events.columns[1:]
        events_list.append(events[event_names].values.T.astype(np.int32))

    return data_list, events_list

In [3]:
def split_train_test_data(data_list, events_list, val_size=2, rand=False):
    # randomly choose val_size time series for validation
    if rand:
        val_ind = np.random.choice(8, size=val_size, replace=False)
    # just use the last two time series for validation
    else:
        val_ind = np.arange(8 - val_size, 8)

    train_data, valid_data = [], []
    train_events, valid_events = [], []
    print(len(data_list))
    # separate the time series into training and validation
    for i in range(1):
        if i == 0:
            train_data.append(data_list[i])
            train_events.append(events_list[i])
        else:
            valid_data.append(data_list[i])
            valid_events.append(events_list[i])

    return train_data, train_events, valid_data, valid_events

In [4]:
# permute the time-windows of all time-series to
# give a shuffled list of time-windows over all
# time series
def get_permuted_windows(series_list, window_size, rand=True):
    series_slices = []
    for i, series in enumerate(series_list):
        slices = get_series_window_slices(series.shape[1],
                                          window_size)
        # need to mark each slice with the series it came from
        for s in slices:
            series_slices.append((i, s))

    # wish to iterate over the windows in random order for training
    if rand:
        random.shuffle(series_slices)
    return series_slices

In [5]:
def get_series_window_slices(num_datapoints, window_size):
    slices = []
    num_windows = num_datapoints - window_size + 1
    for i in range(0, num_windows, 1):
        slices.append(slice(i, i + window_size))

    return slices

In [6]:
def create_iter_funcs_train():
    X = K.placeholder(ndim=3)
    Y = K.placeholder(ndim=2)
    x_batch = K.placeholder(ndim=3)
    y_batch = K.placeholder(ndim=2)

In [7]:
def batch_iterator(bs, W, X, y=None, noisy=False):
    if not W:
        raise StopIteration
    window_size = W[0][1].stop - W[0][1].start
    # total number of batches for this data set and batch size
    N = int((len(W) + bs - 1) / bs)
    for i in range(N):
        Wb = W[i * bs:(i + 1) * bs]

        X_batch_list, y_batch_list = [], []
        # index: which time series to take the window from
        # s:     the slice to take from that time series
        for j, (index, s) in enumerate(Wb):
            X_window = X[index][:, s]
            if y is not None:
                y_window = y[index][:, s][:, -1]
                y_batch_list.append(y_window)

            # this is test data, train data but no action is present,
            # or validation data (don't want noise)
            if y is None or y_window.sum() == 0 or not noisy:
                X_batch_list.append(X_window)
            # this is train data and an action is present, so add noise
            else:
                noise = np.random.normal(
                    0, 0.1, X_window.shape).astype(np.float32)
                X_batch_list.append(X_window + noise)

        # reshape to (batch_size, num_channels, window_size)
        X_batch = np.vstack(X_batch_list).reshape(-1,
                                                  X[0].shape[0], window_size)
        if not y_batch_list:
            y_batch = None
        else:
            y_batch = np.vstack(y_batch_list)

        yield X_batch, y_batch

In [8]:
def build_model(batch_size,num_channels,input_length,output_shape):
    model = Sequential((
    # The first conv layer learns `nb_filter` filters (aka kernels), each of size ``(filter_length, nb_input_series)``.
    # Its output will have shape (None, window_size - filter_length + 1, nb_filter), i.e., for each position in
    # the input timeseries, the activation of each filter at that position.
    Conv1D(nb_filter=8, filter_length=3, activation='relu', input_shape=(batch_size,num_channels)),
    MaxPooling1D(3,2),     # Downsample the output of convolution by 2X.
    Conv1D(nb_filter=16, filter_length=3, activation='relu'),
    MaxPooling1D(3,2),
    Dropout(0.5),
    Dense(32),
    Dense(output_shape),    
    ))
    optimizer = Adadelta()
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [9]:
def build_model_lasagne(batch_size,
                num_channels,
                input_length,
                output_dim,):
    Conv1DLayer = conv.Conv1DLayer
    MaxPool1DLayer = pool.MaxPool1DLayer
    l_in = layers.InputLayer(
        shape=(batch_size, num_channels, input_length),
        name='l_in',
    )

    l_conv1 = Conv1DLayer(
        l_in,
        name='conv1',
        num_filters=8,
        filter_size=3,
        nonlinearity=nonlinearities.rectify,
        W=init.Orthogonal(),
    )

    l_pool1 = MaxPool1DLayer(
        l_conv1,
        name='pool1',
        pool_size=3,
        stride=2,
    )

    l_conv2 = Conv1DLayer(
        l_pool1,
        name='conv2',
        num_filters=16,
        filter_size=3,
        nonlinearity=nonlinearities.rectify,
        W=init.Orthogonal(),
    )

    l_pool2 = MaxPool1DLayer(
        l_conv2,
        name='pool2',
        pool_size=3,
        stride=2,
    )

    l_dropout_dense1 = layers.DropoutLayer(
        #l_pool4,
        l_pool2,
        p=0.5,
    )

    l_dense1 = layers.DenseLayer(
        l_dropout_dense1,
        num_units=32,
        nonlinearity=nonlinearities.rectify,
        W=init.Orthogonal(),
    )

    l_out = layers.DenseLayer(
        l_dense1,
        num_units=output_dim,
        nonlinearity=nonlinearities.sigmoid,
        W=init.Orthogonal(),
    )

    return l_out

In [10]:
def get_current_time():
    return strftime('%Y-%m-%d_%H:%M:%S')

In [11]:
def create_iter_funcs_train_theano(lr, mntm, l_out):
    X = T.tensor3('x')
    y = T.imatrix('y')
    X_batch = T.tensor3('x_batch')
    y_batch = T.imatrix('y_batch')

    train_output = layers.get_output(l_out, X_batch)
    train_loss = T.mean(T.nnet.binary_crossentropy(train_output, y_batch))

    all_params = layers.get_all_params(l_out)
    updates = lasagne.updates.nesterov_momentum(
        train_loss, all_params, lr, mntm)

    train_iter = theano.function(
        inputs=[theano.Param(X_batch),
                theano.Param(y_batch)],
        outputs=[train_loss, train_output],
        updates=updates,
        givens={
            X: X_batch,
            y: y_batch,
        },
    )
    return train_iter

In [12]:
def create_iter_funcs_val_theano(l_out):
    X = T.tensor3('x')
    y = T.imatrix('y')
    X_batch = T.tensor3('x_batch')
    y_batch = T.imatrix('y_batch')

    valid_output = layers.get_output(l_out, X_batch, deterministic=True)
    valid_loss = T.mean(T.nnet.binary_crossentropy(valid_output, y_batch))

    valid_iter = theano.function(
        inputs=[theano.Param(X_batch),
                theano.Param(y_batch)],
        outputs=[valid_loss, valid_output],
        givens={
            X: X_batch,
            y: y_batch,
        },
    )

    return valid_iter

In [13]:
def train_model(window_size, max_epochs, patience):
    #root_dir = join('data', 'nets')
    # the file from which to load pre-trained weights
    #init_file = join(root_dir,
    #                 'subj%d_weights_deep_nocsp_wide.pickle' % (
    #                     4))
    #init_file = join(root_dir,
    #                 'weights_super_deeper.pickle')
    init_file = None
    # the file to which the learned weights will be written
    #weights_file = join(root_dir,
                        #'weights.pickle')
    #temp_weights_file = join(root_dir, 'epoch_%d.pickle')
    train_data, train_events = [], []
    valid_data, valid_events = [], []
    for subj_id in range(1, 2):
        print('loading time series for subject %d...' % (subj_id))
        subj_data_list, subj_events_list = load_subject_train(subj_id)
        print('  creating train and validation sets...')
        subj_train_data, subj_train_events, subj_valid_data, subj_valid_events = \
            split_train_test_data(subj_data_list, subj_events_list,
                                        val_size=2, rand=False)
        train_data += subj_train_data
        train_events += subj_train_events
        valid_data += subj_valid_data
        valid_events += subj_valid_events

    print('using %d time series for training' % (len(train_data)))
    print('using %d time series for validation' % (len(valid_data)))

    print('creating fixed-size time-windows of size %d' % (window_size))
    # the training windows should be in random order
    train_slices = get_permuted_windows(train_data, window_size,
                                                 rand=True)
    valid_slices = get_permuted_windows(valid_data, window_size,
                                                 rand=True)
    print('there are %d windows for training' % (len(train_slices)))
    print('there are %d windows for validation' % (len(valid_slices)))

    #batch_size = 64
    batch_size = 512
    num_channels = 32
    num_actions = 6
    #train_data, valid_data = \
        #utils.preprocess(train_data, valid_data)

    #print('building model %s...' % (
        #sys.modules[build_model.__module__].__name__))
    l_out = build_model_lasagne(None, num_channels,
                        window_size, num_actions)
    print(l_out)
    #all_layers = layers.get_all_layers(l_out)
    #print('this network has %d learnable parameters' %
          #(layers.count_params(l_out)))
    #for layer in all_layers:
        #print('Layer %s has output shape %r' %
              #(layer.name, layer.output_shape))

    if init_file is not None:
        print('loading model weights from %s' % (init_file))
        with open(init_file, 'rb') as ifile:
            src_layers = pickle.load(ifile)
        dst_layers = layers.get_all_params(l_out)
        for i, (src_weights, dst_layer) in enumerate(
                zip(src_layers, dst_layers)):
            print('loading pretrained weights for %s' % (dst_layer.name))
            dst_layer.set_value(src_weights)
    else:
        print('all layers will be trained from random initialization')

    #1r = theano.shared(np.cast['float32'](0.001))
    lr = theano.shared(np.cast['float32'](0.01))
    mntm = 0.9
    print('compiling theano functions...')
    train_iter = create_iter_funcs_train_theano(lr, mntm, l_out)
    valid_iter = create_iter_funcs_val_theano(l_out)

    best_weights = None
    best_valid_loss = np.inf
    best_epoch = 0
    print('starting training for all subjects at %s' % (
        get_current_time()))
    best_weights = None
    best_valid_loss = np.inf
    best_epoch = 0
    print('starting training for all subjects')
    try:
        for epoch in range(max_epochs):
            print('epoch: %d' % (epoch))
            train_losses, training_outputs, training_inputs = [], [], []
            num_batches = (len(train_slices) + batch_size - 1) / batch_size
            t_train_start = time()
            for i, (Xb, yb) in enumerate(
                batch_iterator(batch_size,
                                        train_slices,
                                        train_data,
                                        train_events)):
                t_batch_start = time()
                # hack for faster debugging
                #if i < 70000:
                #    continue
                train_loss, train_output = \
                    train_iter(Xb, yb)
                if np.isnan(train_loss):
                    print('nan loss encountered in minibatch %d' % (i))
                    continue

                train_losses.append(train_loss)
                assert len(yb) == len(train_output)
                for input, output in zip(yb, train_output):
                    training_inputs.append(input)
                    training_outputs.append(output)
                
                batch_duration = time() - t_batch_start
                if i % 10 == 0:
                    eta = batch_duration * (num_batches - i)
                    m, s = divmod(eta, 60)
                    h, m = divmod(m, 60)
                    print('  training...  (ETA = %d:%02d:%02d)\r'
                          % (h, m, s)),
            avg_train_loss = np.mean(train_losses)
            training_inputs = np.vstack(training_inputs)
            training_outputs = np.vstack(training_outputs)
            try:
                train_roc = roc_auc_score(training_inputs, training_outputs)
            except ValueError:
                train_roc = 0
                pass

            train_duration = time() - t_train_start
            print('')
            print('    train loss: %.6f' % (avg_train_loss))
            print('    train roc:  %.6f' % (train_roc))
            print('    duration:   %.2f s' % (train_duration))

            valid_losses, valid_outputs, valid_inputs = [], [], []
            num_batches = (len(valid_slices) + batch_size - 1) / batch_size
            t_valid_start = time()
            for i, (Xb, yb) in enumerate(
                batch_iterator(batch_size,
                                        valid_slices,
                                        valid_data,
                                        valid_events)):
                t_batch_start = time()
                valid_loss, valid_output = \
                    valid_iter(Xb, yb)
                if np.isnan(valid_loss):
                    print('nan loss encountered in minibatch %d' % (i))
                    continue
                valid_losses.append(valid_loss)
                assert len(yb) == len(valid_output)
                for input, output in zip(yb, valid_output):
                    valid_inputs.append(input)
                    valid_outputs.append(output)

                batch_duration = time() - t_batch_start
                if i % 10 == 0:
                    eta = batch_duration * (num_batches - i)
                    m, s = divmod(eta, 60)
                    h, m = divmod(m, 60)
                    #print('  validation...  (ETA = %d:%02d:%02d)\r'
                         # % (h, m, s)),

            # allow training without validation
            if valid_losses:
                avg_valid_loss = np.mean(valid_losses)
                valid_inputs = np.vstack(valid_inputs)
                valid_outputs = np.vstack(valid_outputs)
                try:
                    valid_roc = roc_auc_score(valid_inputs, valid_outputs)
                except ValueError:
                    valid_roc = 0
                    pass
                valid_duration = time() - t_valid_start
                print('')
                print('    valid loss: %.6f' % (avg_valid_loss))
                print('    valid roc:  %.6f' % (valid_roc))
                print('    duration:   %.2f s' % (valid_duration))
            else:
                print('    no validation...')

            # if we are not doing validation we always want the latest weights
            if not valid_losses:
                best_epoch = epoch
                model_train_loss = avg_train_loss
                model_train_roc = train_roc
                model_valid_roc = -1.
                best_valid_loss = -1.
                best_weights = layers.get_all_param_values(l_out)
            elif avg_valid_loss < best_valid_loss:
                best_epoch = epoch
                model_train_roc = train_roc
                model_valid_roc = valid_roc
                model_train_loss = avg_train_loss
                best_valid_loss = avg_valid_loss
                best_weights = layers.get_all_param_values(l_out)

                #temp_file = temp_weights_file % (epoch)
                #print('saving temporary best weights to %s' % (temp_file))
                #with open(temp_file, 'wb') as ofile:
                    #pickle.dump(best_weights, ofile,
                               # protocol=pickle.HIGHEST_PROTOCOL)

            if epoch > best_epoch + patience:
                break
                best_epoch = epoch
                new_lr = 0.5 * lr.get_value()
                lr.set_value(np.cast['float32'](new_lr))
                print('setting learning rate to %.6f' % (new_lr))

    except KeyboardInterrupt:
        print('caught Ctrl-C, stopping training...')

    #with open(weights_file, 'wb') as ofile:
        #print('saving best weights to %s' % (weights_file))
        #pickle.dump(best_weights, ofile, protocol=pickle.HIGHEST_PROTOCOL)
    print('finished training for all subjects at')

    return model_train_loss, best_valid_loss, model_train_roc, model_valid_roc

In [14]:
#print(roc_auc_score([[0,0,0][1,1,1]],[[0,0,0][1,1,1]]))

In [15]:
print(train_model(200,10,1))

loading time series for subject 1...
  creating train and validation sets...
1
using 1 time series for training
using 0 time series for validation
creating fixed-size time-windows of size 200
there are 9801 windows for training
there are 0 windows for validation
all layers will be trained from random initialization
compiling theano functions...


C:\Users\Gebruiker\Anaconda3\lib\site-packages\lasagne\theano_extensions\conv.py:66: UserWarning: The `image_shape` keyword argument to `tensor.nnet.conv2d` is deprecated, it has been renamed to `input_shape`.
  border_mode=border_mode)
C:\Users\Gebruiker\Anaconda3\lib\site-packages\lasagne\layers\pool.py:142: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  padding=(self.pad[0], 0),
C:\Users\Gebruiker\Anaconda3\lib\site-packages\lasagne\layers\pool.py:142: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  padding=(self.pad[0], 0),
C:\Users\Gebruiker\Anaconda3\lib\site-packages\lasagne\layers\pool.py:142: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  padding=(self.pad[0], 0),
C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_lau

starting training for all subjects at 2018-03-12_17:28:08
starting training for all subjects
epoch: 0
  training...  (ETA = 0:17:34)
  training...  (ETA = 0:09:40)

    train loss: 15.570708
    train roc:  0.497541
    duration:   1079.23 s
    no validation...
epoch: 1


C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:141: DeprecationWarning: generator 'batch_iterator' raised StopIteration


  training...  (ETA = 0:19:29)
  training...  (ETA = 0:10:56)

    train loss: 0.397077
    train roc:  0.490750
    duration:   1126.78 s
    no validation...
epoch: 2


...

C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:141: DeprecationWarning: generator 'batch_iterator' raised StopIteration


  training...  (ETA = 0:17:25)
  training...  (ETA = 0:09:08)

    train loss: 0.242395
    train roc:  0.486067
    duration:   1018.97 s
    no validation...
epoch: 9


C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:141: DeprecationWarning: generator 'batch_iterator' raised StopIteration


  training...  (ETA = 0:17:59)
  training...  (ETA = 0:08:45)

    train loss: 0.232717
    train roc:  0.487461
    duration:   1021.11 s
    no validation...
finished training for all subjects at
(0.2327168373720523, -1.0, 0.4874612925089335, -1.0)


C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:141: DeprecationWarning: generator 'batch_iterator' raised StopIteration
